In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os

import lightgbm as lgb

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score

import sys
sys.path.append('../../tools/WOE')
import ppctools_models

pd.set_option('display.max_rows',200)

ModuleNotFoundError: No module named 'lightgbm'

In [18]:
ids = 'ID_code'
tgt = 'target'
feats = ['var_'+str(i) for i in range(200)]

n_splits = 5

params = {
    'objective':'binary',
    'boost':'gbdt',
    'metric':['auc'],
    'learning_rate':0.02,
    'num_leaf':35,
    'subsample':0.8,
    'sub_feature':0.05,
    #'reg_lambda':100,
    'max_bin':340,
    'min_child_samples':50,
    'min_gain_to_split':0.2}

In [3]:
trn = pd.read_csv('../Data/train.csv.zip')
tst = pd.read_csv('../Data/test.csv.zip')

merge_set = ['feats_rank']
for file in merge_set:
    trn = trn.merge(pd.read_csv('../Cache/'+file+'_train.csv'),'inner',ids)
    tst = tst.merge(pd.read_csv('../Cache/'+file+'_test.csv'),'inner',ids)

In [17]:
feats = [var for var in trn if not var in [ids,tgt] and not 'gp' in var and not 'skew' in var]
len(feats)

207

In [14]:
skf = StratifiedKFold(n_splits,True,615)

In [19]:
oof_pred = np.zeros(trn.shape[0])
tst_pred = np.zeros(tst.shape[0])
imp = np.zeros(len(feats))
rounds = 0
for trn_ids,val_ids in skf.split(trn.index,trn[tgt]):
    rounds += 1
    print('Round %d'%rounds)
    trn_lgb = lgb.Dataset(trn.iloc[trn_ids,:][feats],trn.iloc[trn_ids,:][tgt])
    val_lgb = lgb.Dataset(trn.iloc[val_ids,:][feats],trn.iloc[val_ids,:][tgt])
    
    model = lgb.train(params,trn_lgb,10000,val_lgb,early_stopping_rounds =100,verbose_eval = 1000)
    
    oof_pred[val_ids] = model.predict(trn.iloc[val_ids,:][feats])
    
    tst_pred += model.predict(tst[feats])/n_splits
    
    imp += model.feature_importance('gain')/n_splits
print(roc_auc_score(trn[tgt].values,oof_pred))

Round 1
Training until validation scores don't improve for 100 rounds.
[1000]	valid_0's auc: 0.893103
[2000]	valid_0's auc: 0.897065
Early stopping, best iteration is:
[2823]	valid_0's auc: 0.897898
Round 2
Training until validation scores don't improve for 100 rounds.
[1000]	valid_0's auc: 0.894918
[2000]	valid_0's auc: 0.898558
Early stopping, best iteration is:
[2460]	valid_0's auc: 0.899315
Round 3
Training until validation scores don't improve for 100 rounds.
[1000]	valid_0's auc: 0.895416
[2000]	valid_0's auc: 0.899401
Early stopping, best iteration is:
[2746]	valid_0's auc: 0.900296
Round 4
Training until validation scores don't improve for 100 rounds.
[1000]	valid_0's auc: 0.890433
[2000]	valid_0's auc: 0.895206
Early stopping, best iteration is:
[2782]	valid_0's auc: 0.895904
Round 5
Training until validation scores don't improve for 100 rounds.
[1000]	valid_0's auc: 0.893788
[2000]	valid_0's auc: 0.898473
Early stopping, best iteration is:
[2187]	valid_0's auc: 0.898735
0.898

In [8]:
imp = pd.Series(imp,index=feats)
imp.sort_values(ascending=False)

rank_std       59416.150336
gp_7           40618.232425
gp_0           29555.415469
gp_6           19723.972612
gp_9           19649.971969
gp_8           19371.580713
var_53         18773.928839
var_110        18746.128954
var_26         18038.400246
var_174        17867.379878
var_80         17753.113570
var_166        16332.555598
gp_4           16275.463453
gp_2           16037.880629
var_22         15783.559667
var_99         15008.171553
var_165        14540.695891
gp_1           14494.108278
var_108        14254.373380
var_2          13903.095414
var_78         13899.464958
var_190        13745.101636
var_133        13637.174356
var_0          13603.337611
var_44         13116.684414
gp_5           13033.783407
gp_3           12860.880301
var_1          12346.924536
var_179        12178.763384
var_40         12093.837179
var_170        12021.897668
var_164        11911.334688
var_94         11905.002176
var_33         11838.939590
var_9          11787.976177
var_34         11561

In [9]:
submit = pd.DataFrame({ids:tst[ids].values,tgt:tst_pred})
submit.to_csv('../Submission/lgb_rank_%d.csv'%(round(roc_auc_score(trn[tgt].values,oof_pred)*10000,0)),index=False)

In [ ]:
ppctools_models.evaluate_performance(trn[tgt].values,oof_pred)